In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [3]:
#tail -n +2 train.csv | split -l 150000


In [3]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [4]:
splits

['xzbvh',
 'xvv',
 'xzarc',
 'xzfad',
 'xum',
 'xhv',
 'xzdcy',
 'xiz',
 'xxn',
 'xzdlg',
 'xzanz',
 'xzeln',
 'xzagt',
 'xzdle',
 'xng',
 'xzbhs',
 'xzbko',
 'xzcsr',
 'xzckz',
 'xzdme',
 'xjd',
 'xtz',
 'xzaju',
 'xzavo',
 'xzbkq',
 'xyv',
 'xzbpt',
 'xzckn',
 'xzeog',
 'xzapz',
 'xzbdg',
 'xhx',
 'xzcrq',
 'xzbok',
 'xzejd',
 'xec',
 'xzcgi',
 'xhm',
 'xzbmd',
 'xcg',
 'xzeyt',
 'xzcdz',
 'xzbpr',
 'xln',
 'xzbzl',
 'xzcbv',
 'xzeia',
 'xzdoa',
 'xzffk',
 'xzbcd',
 'xzaae',
 'xzcik',
 'xzehd',
 'xzbck',
 'xzcnw',
 'xzdcn',
 'xzcvx',
 'xzcoy',
 'xzfff',
 'xzagf',
 'xjv',
 'xzdln',
 'xzciv',
 'xzahe',
 'xzbia',
 'xzclr',
 'xzdgh',
 'xzcir',
 'xzecg',
 'xzdfm',
 'xth',
 'xzbmj',
 'xzdwi',
 'xzera',
 'xzedf',
 'xhu',
 'xzasm',
 'xzehw',
 'xzboz',
 'xzffm',
 'xzbqb',
 'xzaua',
 'xzbhl',
 'xzdze',
 'xzabi',
 'xzcmm',
 'xzdha',
 'xzahq',
 'xzffq',
 'xzaoy',
 'xzdkz',
 'xzble',
 'xzdjh',
 'xjp',
 'xah',
 'xzevq',
 'xzcsc',
 'xzcmu',
 'xzbgf',
 'xzehx',
 'xhg',
 'xei',
 'xzbay',
 'xzdhm',
 '

In [18]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
split_size = 500


def get_sequence(sequence_idx, sequence_batch_size):
    print("DEBUGGGGG 1")
    splits_chunk = splits[sequence_idx:sequence_idx+sequence_batch_size]
    print("DEBUGGGGG 2")

    training, ys = read_raw_training_data(splits_chunk)
    print("DEBUGGGGG 3")
    new_training, new_ys = fornat_training_data(training, ys)
    print("DEBUGGGGG 4")
    train, Y = swap_axes(new_training, new_ys)
    print("DEBUGGGGG 5")

    return train, Y
    
    
def read_raw_training_data(splits_chunk): 
    training = []
    ys = []

    features = ['acoustic_data','time_to_failure']

    for chunk in splits_chunk:
        path = 'train/%s' % (chunk)
        #print("DEBUGGGGG")
        #print(path)
        chunk_df = pd.read_csv(path, float_precision='round_trip', header=None)
        chunk_df.columns = features
        chunk_df['rolling_100'] = chunk_df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
        chunk_df[features] = StandardScaler().fit_transform(chunk_df[features])
        training.append(chunk_df[features].values)
        ys.append(chunk_df['time_to_failure'].values)

    training = np.array(training)
    ys = np.array(ys)
    
    return training, ys

def fornat_training_data(training, ys):
    new_training = []
    new_ys = []
    for i,sequence in enumerate(training):
        #split each sequence in split_size
        splits = np.array_split(sequence, split_size)
        #print(np.array(splits).shape)
        new_training.append(splits)
        splits_y = np.array_split(ys[i], split_size)
        new_ys.append(splits_y)


    new_training = np.array(new_training)
    new_ys = np.array(new_ys)
    return new_training, new_ys


def swap_axes(new_training,new_ys):
    train = np.swapaxes(new_training,0,1)
    train = train.reshape(-1,300,2)
    
    targets = np.swapaxes(new_ys,0,1)
    
    Y = []
    for target in targets:
        Y.append(np.array(target[:,-1]))

    Y = np.array(Y).reshape(-1)
    
    return train, Y

In [5]:
splits_chunk = splits[0:200]

In [6]:
training = []
ys = []

features = ['acoustic_data','time_to_failure']

from sklearn.preprocessing import StandardScaler,MinMaxScaler 

for chunk in splits_chunk:
    path = 'train/%s' % (chunk)
    chunk_df = pd.read_csv(path, float_precision='round_trip', header=None)
    chunk_df.columns = features
    chunk_df['rolling_100'] = chunk_df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    chunk_df[features] = StandardScaler().fit_transform(chunk_df[features])
    training.append(chunk_df[features].values)
    ys.append(chunk_df['time_to_failure'].values)

training = np.array(training)
ys = np.array(ys)

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

In [8]:
ys.shape

(200, 150000)

In [9]:
training.shape

(200, 150000, 2)

In [8]:
#number of batches must cover a sequence length i.e. 100 * 1500 = 15000, batch size = 1500 

In [8]:
split_size = 500
new_training = []
new_ys = []
for i,sequence in enumerate(training):
    #split each sequence in split_size
    splits = np.array_split(sequence, split_size)
    #print(np.array(splits).shape)
    new_training.append(splits)
    splits_y = np.array_split(ys[i], split_size)
    new_ys.append(splits_y)

    
new_training = np.array(new_training)
new_ys = np.array(new_ys)

In [11]:
new_ys.shape

(200, 500, 300)

In [ ]:
a = np.array([[[1,2,3], [4,5,6]], [[10,20,30], [40,50,60]]])
print(a.shape)
np.array(a[0])[:,-1]

In [ ]:
a = np.array([[1,2,3], [4,5,6]])
a
a.reshape(-1)

In [153]:
np.array(new_ys[0])[:,-1].shape

IndexError: too many indices for array

In [ ]:
new_training.shape

In [9]:
train = np.swapaxes(new_training,0,1)

In [10]:
train = train.reshape(-1,300,2)

In [11]:
targets = np.swapaxes(new_ys,0,1)

In [12]:
Y = []
for target in targets:
    Y.append(np.array(target[:,-1]))
    
Y = np.array(Y).reshape(-1)

In [13]:
Y.shape

(100000,)

In [ ]:
a = np.array([[[[1],[2],[3],[4]],[[5],[6],[7],[8]],[[8],[9],[10],[11]]], [[[10],[20],[30],[40]],[[50],[60],[70],[80]],[[80],[90],[100],[110]]],\
              [[[1000],[2000],[3000],[4000]],[[5000],[6000],[7000],[8000]],[[8000],[9000],[10000],[11000]]] ])
print(a.shape)
a

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(train, Y, test_size=0.1, random_state=42)

In [30]:
from keras.utils import Sequence

class MY_Generator(Sequence):

    def __init__(self, split_filenames, batch_size):
        self.split_filenames = split_filenames
        self.batch_size = batch_size

    #This function computes the number of batches that this generator is supposed to produce. 
    #So, we divide the number of total samples by the batch_size and return that value.    
    def __len__(self):
        return int(np.ceil(len(self.split_filenames) / float(self.batch_size)))

    #Here, given the batch numberidx you need to put together a list that consists of data 
    #batch and the ground-truth (GT). In this example, we read a batch images of size 
    #self.batch and return an array of form[image_batch, GT]
    def __getitem__(self, idx):
        train,Y = get_sequence(idx, self.batch_size)
        print("LOLILOL")
        print(train.shape, Y.shape)

        return (train,Y)

In [14]:
train.shape

(100000, 300, 2)

In [15]:
Y.shape

(100000,)

In [17]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization

BATCH_SIZE=500

dropout=0.2


my_model = Sequential()

my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 3,\
                  stateful=True,batch_input_shape=(BATCH_SIZE,300,2),\
                  dropout=dropout,recurrent_dropout=dropout,
                  ))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=0, verbose=0)
]

history = my_model.fit(train, Y, batch_size=BATCH_SIZE, epochs=100, shuffle=False,
                      #validation_data=(val_data,y_val), callbacks=callbacks
                      )
#my_training_batch_generator = MY_Generator(splits, 100)
        
#my_model.fit_generator(generator=my_training_batch_generator,
                                          #steps_per_epoch=BATCH_SIZE,
                                          #epochs=10,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          #use_multiprocessing=True,
                                          #workers=1,
                                          #max_queue_size=32
                      #)

import matplotlib.pyplot as plt


#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (500, 3)                  72        
_________________________________________________________________
dense_1 (Dense)              (500, 1)                  4         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
100000/100000 [==============================] - 16s 159us/step - loss: 0.7943 - mean_absolute_error: 0.7943
Epoch 2/100
 19000/100000 [====>.........................] - ETA: 12s - loss: 1.3911 - mean_absolute_error: 1.3911

KeyboardInterrupt: 

In [13]:
next(my_training_batch_generator)

TypeError: 'MY_Generator' object is not an iterator